In [67]:
from torch_geometric.datasets import TUDataset
from torch_geometric.loader import DataLoader
from torch_geometric.utils import scatter
import torch.nn as nn
import torch

dataset = TUDataset(root='/tmp/ENZYMES', name='ENZYMES', use_node_attr=True)
loader = DataLoader(dataset, batch_size=32, shuffle=True)

softmax = nn.Softmax(dim=-1)

for step, data in enumerate(loader):
    edge_index, batch = data.edge_index, data.batch

    x = torch.rand(batch.size(0))
    # print(x)

    num_nodes = scatter(batch.new_ones(x.size(0)), batch, reduce='sum')
    print(num_nodes)
    batch_size, max_num_nodes = num_nodes.size(0), int(num_nodes.max())

    cum_num_nodes = torch.cat(
        [num_nodes.new_zeros(1),
         num_nodes.cumsum(dim=0)[:-1]], dim=0)

    index = torch.arange(batch.size(0), dtype=torch.long)
    # print(index)
    # index - cum_num_nodes[batch] 得到的是每个点在各自图中的编号
    index = (index - cum_num_nodes[batch]) + (batch * max_num_nodes)
    # m = batch * max_num_nodes
    # print(index)

    dense_x = x.new_full((batch_size * max_num_nodes, ), -60000.0)
    # print(dense_x.size())
    dense_x[index] = x
    # print(x[0:max_num_nodes*3])
    # print(dense_x[0:max_num_nodes*3])
    dense_x = dense_x.view(batch_size, max_num_nodes)
    # print(dense_x[:3])
    dense_x = softmax(dense_x)
    # print(dense_x[-10:])
    probs, perm = dense_x.sort(dim=-1, descending=True)
    # print(perm)
    cum_probs = torch.cumsum(probs, dim=-1)

    nucleus = (cum_probs < 0.5)+0
    k = torch.count_nonzero(nucleus, dim=1).reshape(-1)
    k = torch.clamp(k, min=1)
    perm = perm + cum_num_nodes.view(-1, 1)
    # print(perm)
    perm = perm.view(-1)

    index = torch.cat([
        torch.arange(k[i], device=x.device) + i * max_num_nodes
        for i in range(batch_size)
    ], dim=0)

    perm = perm[index]

    break

tensor([23, 44, 44, 14, 23, 19, 39, 14, 13, 33, 25, 41, 37, 40, 54, 44, 31, 22,
        42,  5, 24, 48, 38, 16, 23, 19, 35, 46, 51, 13, 18, 14])
tensor([  0,  23,  67, 111, 125, 148, 167, 206, 220, 233, 266, 291, 332, 369,
        409, 463, 507, 538, 560, 602, 607, 631, 679, 717, 733, 756, 775, 810,
        856, 907, 920, 938])
54
tensor([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,   10,   11,
          12,   13,   14,   15,   16,   17,   18,   19,   20,   21,   22,   54,
          55,   56,   57,   58,   59,   60,   61,   62,   63,   64,   65,   66,
          67,   68,   69,   70,   71,   72,   73,   74,   75,   76,   77,   78,
          79,   80,   81,   82,   83,   84,   85,   86,   87,   88,   89,   90,
          91,   92,   93,   94,   95,   96,   97,  108,  109,  110,  111,  112,
         113,  114,  115,  116,  117,  118,  119,  120,  121,  122,  123,  124,
         125,  126,  127,  128,  129,  130,  131,  132,  133,  134,  135,  136,
         137,  138,  139,  